In [20]:
from modeling_showui import ShowUIForConditionalGeneration
from processing_showui import ShowUIProcessor
# from transformers import AutoProcessor
from configuration_showui import Qwen2VLConfig
from qwen_vl_utils import process_vision_info
import time 

In [21]:
model_path = "/data/data1/syc/intern/wanshan/models/Qwen2-VL-2B-Instruct"
device = 'cuda'
uigraph_train = True        # Enable ui graph during training
uigraph_test = True         # Enable ui graph during inference
uigraph_diff = 1            # Pixel difference used for constructing ui graph
uigraph_rand = False        # Enable random graph construction 
# 2. Graph -> Mask 
uimask_pre = True           # Prebuild patch selection mask in the preprocessor (not in model layers) for efficiency
uimask_ratio = 0.5          # Specify the percentage of patch tokens to skip per component
uimask_rand = False         # Enable random token selection instead of uniform selection

In [22]:
min_pixel = 28 * 4
max_pixel = 28 * 30
processor = ShowUIProcessor.from_pretrained(
        model_path)

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "./chrome.png",
                # "resized_height": 28 * 30,
                # "resized_width": 28 * 30,
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to(device)

(1, 39, 69)


In [23]:
for key in inputs.keys():
    print(key)

input_ids
attention_mask
patch_pos
pixel_values
image_grid_thw
patch_assign
patch_assign_sep
patch_assign_len


In [24]:
img_grid_thw = inputs['image_grid_thw']
img_grid_thw

tensor([[  1,  78, 138]], device='cuda:0')

In [25]:
patch_pos = inputs['patch_pos']
patch_pos.shape

torch.Size([1, 2716])

In [26]:
select_mask = inputs['select_mask']
select_mask.shape

KeyError: 'select_mask'

In [17]:
patch_assign = inputs['patch_assign']
print(patch_assign[:20])

tensor([ 0,  1,  2,  3,  4,  5,  6,  6,  6,  6,  6,  7,  8,  9, 10, 10, 10, 10,
        10, 10], device='cuda:0')


In [18]:
patch_assign_len = inputs['patch_assign_len']
patch_assign_len

tensor([257], device='cuda:0')

In [9]:
patch_pos = inputs['patch_pos']
patch_pos[patch_pos != -1]

tensor([  0,   1,   2,  ..., 255, 256, 225], device='cuda:0')

In [10]:
inputs['patch_assign_len']

tensor([257], device='cuda:0')